Wakacje są blisko, wszyscy już myślą o wyjazdach - ale jak tu dobrze ocenić hotel, żeby jak najbardziej się nim cieszyć?

Zadanie polega na stworzeniu modelu, który będzie klasyfikował liczbę gwiazdek przyznanych hotelowi (rating, klasa = liczba_gwiazdek - 1) na podstawie recenzji, jaką otrzymał (review).
Wszelkie chwyty dozwolone :) Mogą Państwo przewidywać klasy w oparciu o klasyczną metodę bag of words, inne metody reprezentacji dokumentów (np. TF-IDF), korzystać z embeddingów, gotowych modeli językowych itd. Do dyspozycji mają Państwo dane treningowe, oczekuję od Państwa pliku csv z wygenerowanymi predykcjami.
Proszę zwrócić uwagę na fakt, że jest to problem klasyfikacji wieloklasowej z mocno niezbalansowanym zbiorem danych!
Proszę także Państwa o przetestowanie kilku (co najmniej trzech) podejść do klasyfikacji - mogą (ale nie muszą!) być to gotowe modele, najlepiej o różnych architekturach.

Bardzo proszę, żeby zwrócili mi Państwo archiwum zip (wystarczy jedna osoba z zespołu), proszę też o zastosowanie się do instrukcji:
- Archiwum i wszystkie pliki powinny być nazwane {poniedzialek/piatek}_nazwisko1_nazwisko2
- W archiwum proszę (bez zbędnych podfolderów!) umieścić pliki ze swoim kodem i testowe predykcje nazwane zgodnie z sekwencją  {poniedzialek/piatek}_nazwisko1_nazwisko2.csv
- Testowe predykcje powinny mieć kolejność zgodną z kolejnością sekwencji, do których się odnoszą w zbiorze testowym. Plik csv nie powinien mieć nagłówka ani indeksów.

Proszę o dokładne dokumentowanie wykonanych eksperymentów!

Uwaga: proszę dokładnie sprawdzić swoje rozwiązania i predykcje. W związku z końcem semestru, nie będzie możliwości dosyłania poprawek.

In [104]:
# import zipfile

# # Specify the path to the zip file
# zip_file_path = '/content/data.zip'

# # Specify the directory to extract the contents to
# extract_dir = '/content/data/'

# # Extract the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)

# # Check the extracted directory structure
# !ls {extract_dir}

In [105]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import numpy as np
import pandas as pd
import nltk
from nltk import ngrams
from tqdm import tqdm

In [106]:
DATA_PATH = './data'
MODEL_PATH = './models'

In [107]:
train_data = pd.read_csv(f'{DATA_PATH}/train_data.csv')
test_data = pd.read_csv(f'{DATA_PATH}/test_data.csv')

### Continuous Bag Of Words

In [108]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
EPOCHS_CBOW = 10

In [109]:
reviews = train_data['review']
ngrams = []
for review in reviews:
    review = review.lower().split()
    ngram = [
        (
            [review[i - j - 1] for j in range(CONTEXT_SIZE)] + [review[i + j + 1] for j in range(CONTEXT_SIZE)],
            review[i]
        )
        for i in range(CONTEXT_SIZE, len(review)-CONTEXT_SIZE)
        ]
    ngrams.extend(ngram)

vocab = set([word for review in reviews for word in review.lower().split()])

word_to_ix = {word: i for i, word in enumerate(vocab)}

ngrams

[(['not', 'location', 'excellent', 'hotel'], 'palace'),
 (['palace', 'not', 'hotel', 'booke'], 'excellent'),
 (['excellent', 'palace', 'booke', 'dthe'], 'hotel'),
 (['hotel', 'excellent', 'dthe', 'hotel'], 'booke'),
 (['booke', 'hotel', 'hotel', 'nh'], 'dthe'),
 (['dthe', 'booke', 'nh', 'hotel'], 'hotel'),
 (['hotel', 'dthe', 'hotel', 'site'], 'nh'),
 (['nh', 'hotel', 'site', 'attractive'], 'hotel'),
 (['hotel', 'nh', 'attractive', 'rate,'], 'site'),
 (['site', 'hotel', 'rate,', 'room'], 'attractive'),
 (['attractive', 'site', 'room', 'spacious'], 'rate,'),
 (['rate,', 'attractive', 'spacious', 'quiet,'], 'room'),
 (['room', 'rate,', 'quiet,', 'clean,'], 'spacious'),
 (['spacious', 'room', 'clean,', 'hotel'], 'quiet,'),
 (['quiet,', 'spacious', 'hotel', 'bit'], 'clean,'),
 (['clean,', 'quiet,', 'bit', 'unpersonal'], 'hotel'),
 (['hotel', 'clean,', 'unpersonal', 'big,'], 'bit'),
 (['bit', 'hotel', 'big,', 'breakfast'], 'unpersonal'),
 (['unpersonal', 'bit', 'breakfast', 'tremendous,'], 

In [110]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(2* context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [111]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in tqdm(range(EPOCHS_CBOW)):
    total_loss = 0
    for context, target in ngrams:

        # Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1} - Total loss: {total_loss}')
    losses.append(total_loss)

  0%|          | 0/10 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
model_path = f'{MODEL_PATH}/ngram_model.pth'
torch.save(model.state_dict(), model_path)

### Embedding? 

### LSTM

### Gotowy model jakikolwiek